###  PYTHON REBOUND ORBITAL INTEGRATOR NOTEBOOK  ###

THIS SCRIPT INTEGRATES ANY SYSTEM THAT CONSISTS OF PLANETS AND PARTICLES ORBITING A SINGLE STAR.\
ITS OBJECTIVE IS TO MAKE EASIER THE NUMERICAL INTEGRATION SETUP AND TO AVOID WRITING UNNECESSARY AND REPETITIVE CODE.

THERE ARE 3 MODES OF OPERATION WHICH ARE DESCRIBED BELOW:
* IN THE NORMAL MODE (MODE 0) THE PROGRAM INTEGRATES A SYSTEM WHOSE INITIAL CONDITIONS ARE SET IN THE SCRIPT (CELL 3).
* IN THE MODE 1 THIS INFORMATION IS READ FROM AN INPUT FILE. ADDITIONALLY, IF THE "EVORB" COMPATIBILITY IS ENABLED, READS DATA FROM THE SAME INPUT FILE USED FOR EVORB15, RESPECTING ITS FORMAT AND WRITES THE OUTPUT IN THE SAME FORMAT EVORB15 DOES.
* IN THE MODE 2 NO INTEGRATION IS PERFORMED BUT IS LOADED FROM A FILE. THIS MODE IS USEFUL TO CREATE (AND SAVE) PLOTS AND/OR TO SHARE DATA WITH ANOTHER NOTEBOOK. 

AFTER THE INTEGRATION, IT PLOTS THE RESULTS AND SAVE THEM (IF ACTIVATED).\
IT ALSO SAVES THE DATA IN COLUMN FORMATTED FILES (IF ACTIVATED).\
IT CAN PLOT/SAVE DATA IN A SINGLE FIGURE/FILE OR IN MULTIPLE FIGURES/FILES.\
ALL THESE CONFIGURATIONS ARE DEFINED BY THE USER IN CELL 2.

¡ATENTION!: DO NOT MODIFY NOTHING IN THIS SCRIPT EXCEPT THE USER ASKED INFORMATION IN CELLS 2 AND 3.

BUG REPORT/SUGGESTIONS/WHATEVER ELSE: juan.pons.93@gmail.com

In [1]:
# Libraries used:
import rebound
import reboundx
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from math import *
from IPython.display import display, clear_output
from ipywidgets import IntProgress
from IPython.display import display
import time
import sys 
import os

# Some general configurations for plots:
plt.style.use('seaborn-whitegrid')
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 19
# mpl.rcParams['lines.linewidth'] = 11
# mpl.rcParams['lines.markersize'] = 11

In [2]:
# *** GENERAL CONFIGURATIONS ***

# Acronyms: ICs = Initial conditions
#           Int. = Integration

# ** MODES OF OPERATION **
# Select it with flag MODO:
# MODO = 0 --> ICs are set in the next cell.
# MODO = 1 --> ICs are read from file (the format depends on EVORB15 flag). 
# MODO = 2 --> An integration is loaded from file. 
MODO = 1

# ** DIRECTORIES **
# (In linux use "/" to separate folders whereas in Windows use "\\")
# Common path:
COMUN = "/home/juan/Escritorio/TESIS/"
# Data path (data is saved/loaded_from here):
dir_data = COMUN + "PROGRAMAS/Hamiltoniano/Hasigma_planetario/PARTICULARES/HD31527/bcd/"
#dir_data = COMUN + "PROGRAMAS/REBOUND/COPLANAR/Restricto/3-2/ep=0.3/2/"
# Plot path (plots are saved here):
dir_plot = dir_data

# ** FILES **
# Files have the following format if MULTIPLE = 2 (see below): name_N.format being 
# where "name" is the name defined below for data and plot files
#       "N" the number of planet/particle and
#       "format" the format given in the following setup block.
# Data general names: 
file_data     = "objetos"
file_data_pla = "planeta"
file_data_par = "particula"
# ICs file name:
file_data_in = "inicond"    
# Plot general names:
file_plot_pla = "planeta" 
file_plot_par = "particula" 
file_plot_oth = "otros" # Used for example for resonant angles or other special quantities (NO IMPLEMENTED YET)

# ** FORMATS **
form_data = "txt" # Supported: txt, dat, ...
form_plot = "jpg" # Supported: jpg, png, ...

# ** OTHER (IMPORTANT) SETUP **
# Angle's domain?
# ANGLE = 0 -> (0, 360).
# ANGLE = 1 -> (-180, 180). 
# ANGLE = 2 -> (0, 2pi).
# ANGLE = 3 -> (-pi, pi).
ANGLE     = 0
# Show status before and after integration?
STATUS = False
# Save simulation snapshot binary? (useful for very long simulations to prevent complete int. lost)
SNAPSHOT = False
# Load orbital elements to sharable variables for using them in another notebook?
COMPARAR  = True
# Save integration data and plots (MODO = 1 only can save plots)
SAVE_data = False
SAVE_plot = False
# Image format and quality (in dpi)
cal       = 100
# Produce multiple output files/plots?    # (NO IMPLEMENTED YET) 
# MULTIPLE = 0 -> Unique output file. Separated planet's plot and particle's plot.
# MULTIPLE = 1 -> Separated planet's file/plot and particle's file/plot.
# MULTIPLE = 2 -> A file and a plot for each object.
MULTIPLE  = 1   
# EVORB15 Compatibility? (this option changes the input expected format and output format)
EVORB     = False
# Plot other (special/particular) plots?
OTROS = False

In [3]:
# *** INTEGRATION CONFIGURATION AND SYSTEM ICs ***

# ** INTEGRATION SETUP **

# Units used (default = ('yrs', 'AU', 'Msun')):
UNIDADES = ('yrs', 'Au', 'Msun')
# Allowed for...
# ...time:     Hr, Yr, Jyr, Sidereal_yr, Yr2pi, Kyr, Myr, Gyr.
# ...distance: M, Cm, Km, AU.
# ...mass:     Kg, Msun, Mmercury, Mvenus, Mearth, Mmars, Mjupiter, Msaturn, Muranus, Mpluto.
# Important notes:
#    1) All radius must be in Km when Au used for distance.
#    2) When RELAT = True, use default units.

# Integration time setup:
Ttot  = 1e6   # total int. time
dT    = 0.01  # int. step (while no encounter occurs)
d_out = 1e3   # output step (always >= dT)

# Reference frame? (supported: "Astro", "Bary", "Poincare", "Jacobi").
FRAME = "Astro"


# ** EXTRA PHYSICAL EFFECTS **
# Consider relativity effects?
RELAT = False # (Only works correctly when UNIDADES = ('yrs', 'AU', 'Msun'))


# ** INITIAL CONDITIONS SETUP **

# Orbital element's notation:
# a = semi-major axis; e = eccentriciy; I = inclination; 
# O = longitude of the ascending node; w = argument of pericenter; M = mean anomaly.

# Subindex notation: Planet = p; Particle = nothing; Initial = i

# Central body mass and radius (in km)
m0 = 1   
r0 = 696000 # (Sun radius = 696000 km ~ 0.00465247263 au)

# In MODO = 0 you must indicate in arrays the planet's mass/radius and the object's initial elements
if (MODO == 0):

    # Masses and radius of planets
    mp = [0.0001] 
    rp = [35000] # (in km)

    # Planet(s) initial elements:
    api = [5.2]
    epi = [0.3]
    Ipi = [0]
    Opi = [0]
    wpi = [0] 
    Mpi = [0]

    # Particle(s) initial elements:
    ai = [3.967946, 3.967946*0.997]
    ei = [0.7, 0.7]
    Ii = [0, 0]
    Oi = [0, 0]
    wi = [0, 0]
    Mi = [0, 0]

# *** FROM HERE ON, DO NOT MODIFY THE CODE ***
    
# In case MODO = 1, the next cell will process input files.

In [4]:
# THIS CELL CHECKS FOR ANY WRONG VALUE IN THE VARIABLES ENTERED BY THE USER

modo_val = [0, 1, 2]
if (MODO not in modo_val):
    print('MODO flag value is incorrect. Please, define it as 0, 1 or 2.')
    sys.exit() # exit Python

ang_val = [0, 1, 2, 3]
if (ANGLE not in ang_val):
    print('ANGLE value is incorrect. Valid values are 0, 1, 2 and 3.')
    sys.exit() # exit Python
    
if (d_out < dT)and(MODO != 2):
    d_out = dT
    print('d_out was re-defined as dT.')
    
frame_val = ["Astro", "Bary", "Poincare", "Jacobi"]
if (FRAME not in frame_val)and(MODO != 2):
    print('Invalid reference frame.')
    sys.exit() # exit Python

# Files names with extension included
file_data_ext = file_data + '.' + form_data
file_data_pla_ext = file_data_pla + '.' + form_data
file_data_par_ext = file_data_par + '.' + form_data

In [5]:
# In MODO = 1 the ICs are read from file. Expected format depends on flag EVORB.
if (MODO == 1):
    
    # Initiallize arrays
    api = []
    epi = []
    Ipi = []
    Opi = []
    wpi = [] 
    Mpi = []
    mp  = []
    rp  = []
    ai  = []
    ei  = []
    Ii  = []
    Oi  = []
    wi  = [] 
    Mi  = []
    
    if EVORB: 
        
        print('Mode 1 activated with EVORB15 compatibility.')

        # This piece of code is to find parameters from datos.dat (input file for numerical integration)
        string1 = "INDICE CORRECCION RELATIVISTA"
        string2 = "MASA CENTRAL"
        string3 = "TIEMPO MAXIMO DE LA SIMULACION"
        string4 = "INTERVALO DE ESCRITURA"
        string5 = "PASO DE INTEGRACION"
        string6 = "NUMERO DE PLANETAS"
        string7 = "elementos J2000 de los planetas"
        string8 = "NUMERO DE PARTICULAS"
        string9 = "elementos J2000 de ESAS particulas"

        dir_evorb = dir_data
        datos_evorb = "datos.dat"
        # opening a text file
        file = open(dir_evorb + datos_evorb, "r", encoding = "ISO-8859-1")

        # setting flag and cont to 0
        flag = 0
        cont = 0

        # Loop through the file line by line
        for line in file:

            if (flag == 1):
                relat_flag = int(float(line))
                if (relat_flag == 0):
                    RELAT = False         
                if ((relat_flag == 1) or (relat_flag == 2)):
                    RELAT = True
                flag = 0

            if (flag == 2):
                masaradio = [float(x) for x in line.split()]   
                m0 = masaradio[0]
                r0 = masaradio[1]
                flag = 0

            if (flag == 3):
                Ttot = float(line)
                flag = 0

            if (flag == 4):
                d_out = float(line)
                flag = 0

            if (flag == 5):
                dT = float(line)
                flag = 0

            if (flag == 6):
                Npla = int(float(line))
                flag = 0

            if (flag == 7):
                cont = cont + 1
                elem = [float(x) for x in line.split()] 
                api.append(elem[1])
                epi.append(elem[2])
                Ipi.append(elem[3])
                Opi.append(elem[4])
                wpi.append(elem[5]) 
                Mpi.append(elem[6])
                mp.append(elem[7])
                rp.append(elem[8])
                if (cont == Npla):
                    flag = 0
                    cont = 0

            if (flag == 8):
                Npar = int(float(line))
                flag = 0

            if (flag == 9):
                cont = cont + 1
                elem = [float(x) for x in line.split()] 
                ai.append(elem[1])
                ei.append(elem[2])
                Ii.append(elem[3])
                Oi.append(elem[4])
                wi.append(elem[5]) 
                Mi.append(elem[6])
                if (cont == Npar):
                    flag = 0


            # checking if string is present in line or not
            if string1 in line:
                flag = 1
            if string2 in line:
                flag = 2
            if string3 in line:
                flag = 3
            if string4 in line:
                flag = 4
            if string5 in line:
                flag = 5
            if string6 in line:
                flag = 6
            if string7 in line:
                flag = 7
                cont = 0
            if string8 in line:
                flag = 8
            if string9 in line:
                flag = 9
                cont = 0
        # end of for        
        
    # Simple input file expected        
    else:
   
        print('Mode 1 activated. Expected file with format: mass a e I O w M')
    
        file_data_in_ext = file_data_in + '.' + form_data
        file = open(dir_data + file_data_in_ext, "r", encoding = "ISO-8859-1")
        
        next(file) # Skip header
                 
        # Loop through the file line by line        
        for line in file:
            
            elem = [float(x) for x in line.split()] 
            
            if (len(elem)>0):
            # Decide if is planet or particle
                if (elem[0]!=0):
                    mp.append(elem[0])
                    rp.append(10000) # for now fix r = 10000 km
                    api.append(elem[1])
                    epi.append(elem[2])
                    Ipi.append(elem[3])
                    Opi.append(elem[4])
                    wpi.append(elem[5]) 
                    Mpi.append(elem[6])
                else:
                    ai.append(elem[1])
                    ei.append(elem[2])
                    Ii.append(elem[3])
                    Oi.append(elem[4])
                    wi.append(elem[5]) 
                    Mi.append(elem[6])

# Npla, Npar and Nobj are calculated in the same way for modes 0 and 1:                    
if (MODO != 2):                    
    Npla = len(api)    # Number of planets
    Npar = len(ai)     # Number of particles
    Nobj = Npla + Npar # Number of objects                    

Mode 1 activated. Expected file with format: mass a e I O w M


In [6]:
# In MODO = 2 the numerical integrations are read from file.
if (MODO == 2):
    
    # 2 dataframes, one for planets, another for particles
    Dpla = pd.read_csv(dir_data + file_data_pla_ext, delimiter=r"\s+")
    Dpar = pd.read_csv(dir_data + file_data_par_ext, delimiter=r"\s+")

    # Number of objects   
    Npla = np.max(Dpla['N'].values)
    Npar = np.max(Dpar['N'].values) - Npla
    Nobj = Npla + Npar   
   
    # Max time
    tpla = Dpla['t'].values
    tpar = Dpar['t'].values
    tmax1 = np.max(tpla)
    tmax2 = np.max(tpar)
    tmax = max([tmax1, tmax2])
    
    # Time step
    d_out = tpla[Npla]-1  # ese -1 esta turbio...
    
    # output total points
    Nout   = int(tmax/d_out) 
    
    # Construct time vector
    times = np.linspace(0, Ttot, Nout)
    t = times
    
    # Elements initialization
    ap = np.empty([Npla,Nout])
    ep = np.empty([Npla,Nout])
    Ip = np.empty([Npla,Nout])
    Op = np.empty([Npla,Nout])
    wp = np.empty([Npla,Nout])
    Mp = np.empty([Npla,Nout])
    a  = np.empty([Npar,Nout])
    e  = np.empty([Npar,Nout])
    I  = np.empty([Npar,Nout])
    O  = np.empty([Npar,Nout])
    w  = np.empty([Npar,Nout])
    M  = np.empty([Npar,Nout])
    ap[:] = np.NaN
    ep[:] = np.NaN
    Ip[:] = np.NaN
    Op[:] = np.NaN
    wp[:] = np.NaN
    Mp[:] = np.NaN
    a[:]  = np.NaN
    e[:]  = np.NaN
    I[:]  = np.NaN
    O[:]  = np.NaN
    w[:]  = np.NaN
    M[:]  = np.NaN

#     for i,time in enumerate(times):
    # add planet(s) elements:
    for j in range(Npla):
        ap[j][:] = Dpla[Dpla.N == j+1]['ap'].values
        ep[j][:] = Dpla[Dpla.N == j+1]['ep'].values
        Ip[j][:] = Dpla[Dpla.N == j+1]['Ip'].values
        Op[j][:] = Dpla[Dpla.N == j+1]['Op'].values
        wp[j][:] = Dpla[Dpla.N == j+1]['wp'].values
        Mp[j][:] = Dpla[Dpla.N == j+1]['Mp'].values
    # add particle(s) elements:
    for j in range(Npar):    
        a[j][:] = Dpar[Dpar.N == j+1+Npla]['a'].values
        e[j][:] = Dpar[Dpar.N == j+1+Npla]['e'].values
        I[j][:] = Dpar[Dpar.N == j+1+Npla]['I'].values
        O[j][:] = Dpar[Dpar.N == j+1+Npla]['O'].values
        w[j][:] = Dpar[Dpar.N == j+1+Npla]['w'].values
        M[j][:] = Dpar[Dpar.N == j+1+Npla]['M'].values
        
    # Print some info
    print('Numerical integration was loaded from file.')
    print(Npla,'planet(s) and ', Npar,'particle(s) present in the integration.')

In [7]:
# INTEGRATION PRE-SETUP

if (MODO!=2):
    # Convert ICs lists to numpy arrays
    api = np.asarray(api)
    epi = np.asarray(epi)
    Ipi = np.asarray(Ipi)
    Opi = np.asarray(Opi)
    wpi = np.asarray(wpi)
    Mpi = np.asarray(Mpi)
    ai =  np.asarray(ai)
    ei =  np.asarray(ei)
    Ii =  np.asarray(Ii)
    Oi =  np.asarray(Oi)
    wi =  np.asarray(wi)
    Mi =  np.asarray(Mi)

    # Create sim
    sim = rebound.Simulation()

    # Integration algorithm selection:
    # sim.integrator = "ias15"
    # sim.integrator = "whfast"
    sim.integrator = "mercurius" # whfast + ias15 (in encounters)
    sim.ri_mercurius.hillfac = 4. # Hill radius for mercurius encounter criteria

    # Reference frame:
    sim.ri_mercurius.coordinates = 'democraticheliocentric'# ~ Poincare
    if (FRAME == "Jacobi"):
        sim.ri_whfast.coordinates = 'jacobi' # (Default)
    # sim.ri_whfast.coordinates = 'whds' (more details in paper?)
    if (FRAME == "Astro"):
        sim.move_to_hel() # Move to heliocentric frame
    if (FRAME == "Bary"):
        sim.move_to_com() # Move to center-of-mass frame

    # Provide unit system and time step:
    sim.units = UNIDADES
    #print("G = {0}.".format(sim.G))
    sim.dt = dT
    #sim.t = Ttot
    Nout   = int(Ttot/d_out) # output total points

    # Improve performance and accuaracy:
    # sim.ri_whfast.safe_mode = 0
    # sim.ri_whfast.corrector = 11 # Only in Jacobi

    # Some factors definitions:
    KG2 = sqrt(sim.G)
    G2R   = pi/180
    R2G   = 180/pi
    Km2Au = 1/149597870.7
    Au2Km = 149597870.7

    # Convert radius
    r0 = r0*Km2Au
    rp = np.asarray(rp)
    rp = rp*Km2Au

    # OBJECTS DEFINITION:
    star = sim.add(m=m0, r=r0)       

    if ((ANGLE == 0) or (ANGLE == 1)):
        Ipi = Ipi*G2R
        wpi = wpi*G2R
        Opi = Opi*G2R
        Mpi = Mpi*G2R
        Ii  = Ii*G2R
        wi  = wi*G2R
        Oi  = Oi*G2R
        Mi  = Mi*G2R

    pla = np.empty(Npla)# ,dtype=object   a e i nodo argper anomedia 
    for i in range(Npla):
        pla[i] = sim.add(m=mp[i], r=rp[i], a=api[i], e=epi[i], inc=Ipi[i], Omega=Opi[i], omega=wpi[i], M=Mpi[i])

    par = np.empty(Npar)
    for i in range(Npar):
        par[i] = sim.add(m=0, a=ai[i], e=ei[i], inc=Ii[i], Omega=Oi[i], omega=wi[i], M=Mi[i])

    # Put all the objects in the "partciles" variable
    particles = sim.particles

    # This is necessary also after setting up particles
    if (FRAME == "Astro"):
        sim.move_to_hel() # Move to heliocentric frame
    if (FRAME == "Bary"):
        sim.move_to_com() # Move to center-of-mass frame

    # Relativity effects
    if (RELAT):
        from reboundx import constants
        rebx = reboundx.Extras(sim)
        gr = rebx.load_force("gr_full")
        rebx.add_force(gr)
        gr.params["c"] = (constants.C)*(2*pi)
        print('General relativity corrections activated.')

    # Inform status before int.
    if (STATUS):
        sim.status()

In [8]:
# REBOUND NUMERICAL INTEGRATION

if (MODO!=2):
    # Variables initialization:
    times = np.linspace(0, Ttot, Nout)
    t = times

    ap = np.empty([Npla,Nout])
    ep = np.empty([Npla,Nout])
    Ip = np.empty([Npla,Nout])
    Op = np.empty([Npla,Nout])
    wp = np.empty([Npla,Nout])
    Mp = np.empty([Npla,Nout])
    a  = np.empty([Npar,Nout])
    e  = np.empty([Npar,Nout])
    I  = np.empty([Npar,Nout])
    O  = np.empty([Npar,Nout])
    w  = np.empty([Npar,Nout])
    M  = np.empty([Npar,Nout])
    ap[:] = np.NaN
    ep[:] = np.NaN
    Ip[:] = np.NaN
    Op[:] = np.NaN
    wp[:] = np.NaN
    Mp[:] = np.NaN
    a[:]  = np.NaN
    e[:]  = np.NaN
    I[:]  = np.NaN
    O[:]  = np.NaN
    w[:]  = np.NaN
    M[:]  = np.NaN

    # Progress bar definition:
    print('Progress:')
    barra = IntProgress(min=0, max=Nout) # instantiate the bar
    display(barra) # display the bar

    # Integration carried out in (Ttot/Nout) time lapsus in order to save elements:
    for i,time in enumerate(times):
        # Increment integration by "time"
        sim.integrate(time, exact_finish_time=0)
        # Save planet(s) elements:
        for j in range(Npla):
            ap[j][i] = particles[j+1].a
            ep[j][i] = particles[j+1].e
            Ip[j][i] = (particles[j+1].inc  ) % (2*pi)
            Op[j][i] = (particles[j+1].Omega) % (2*pi)
            wp[j][i] = (particles[j+1].omega) % (2*pi)
            Mp[j][i] = (particles[j+1].M    ) % (2*pi)
        # Save particle(s) elements:
        for j in range(Npar):    
            a[j][i]  = particles[j+1+Npla].a
            e[j][i]  = particles[j+1+Npla].e
            I[j][i]  = (particles[j+1+Npla].inc  ) % (2*pi)     
            O[j][i]  = (particles[j+1+Npla].Omega) % (2*pi)    
            w[j][i]  = (particles[j+1+Npla].omega) % (2*pi)    
            M[j][i]  = (particles[j+1+Npla].M    ) % (2*pi)
        # Signal to increment the progress bar   
        barra.value += 1 

        if ((SNAPSHOT) and ((time % int(Ttot/10)) == 0)):
            sim.save("snapshot.bin")  

    print('The integration has finished.')

    # Inform status after int.
    if (STATUS):
        sim.status()

# END OF NUMERICAL INTEGRATION

Progress:


IntProgress(value=0, max=1000)

KeyboardInterrupt: 

In [ ]:
# INTEGRATION POST-PROCESSING


# SOME ANGLES OPERATIONS:
# If ANGLE = 0 => The angles: (0, 360). If ANGLE = 1 => The angles: (-180,180). 
# If ANGLE = 2 => The angles: (0, 2pi). If ANGLE = 3 => The angles: (-pi,pi).

if ((ANGLE == 1) or (ANGLE == 3)):
    Ip = np.where(Ip>pi, Ip - 2*pi, Ip)
    wp = np.where(wp>pi, wp - 2*pi, wp)
    Op = np.where(Op>pi, Op - 2*pi, Op)
    Mp = np.where(Mp>pi, Mp - 2*pi, Mp)
    I  = np.where(I >pi, I  - 2*pi, I)
    w  = np.where(w >pi, w  - 2*pi, w)
    O  = np.where(O >pi, O  - 2*pi, O)
    M  = np.where(M >pi, M  - 2*pi, M)

ang_unit = '(rad)'
if ((ANGLE == 0) or (ANGLE == 1)):
    ang_unit = '(°)'
    if (MODO!=2):
        Ip = Ip*R2G
        wp = wp*R2G
        Op = Op*R2G
        Mp = Mp*R2G
        I  = I*R2G
        w  = w*R2G
        O  = O*R2G
        M  = M*R2G 

if (ANGLE == 0):
    angle_yticks = np.arange(0,450,90)
if (ANGLE == 1):
    angle_yticks = np.arange(-180,270,90)
if (ANGLE == 2):
    angle_yticks = np.arange(0,(2+1/2)*pi,pi/2)
if (ANGLE == 3):
    angle_yticks = np.arange(-pi,(1+1/2)*pi,pi/2)

In [ ]:
# PLANET'S PLOTS
fig, axs = plt.subplots(6, 1, figsize = (16, 14), sharex = 'col', sharey = False)

# Plot: ap(t), ep(t) and anglesp(t).
for i in range(Npla):
    axs[0].plot(t, ap[:][i], marker = '.', ms = 1, ls = 'None')
    axs[1].plot(t, ep[:][i], marker = '.', ms = 1, ls = 'None')
    axs[2].plot(t, Ip[:][i], marker = '.', ms = 1, ls = 'None')
    axs[3].plot(t, Op[:][i], marker = '.', ms = 1, ls = 'None')
    axs[4].plot(t, wp[:][i], marker = '.', ms = 1, ls = 'None')
    axs[5].plot(t, Mp[:][i], marker = '.', ms = 1, ls = 'None')

# Labels y ticks:
axs[0].set(ylabel = '$a_p$ (au)')#, ylim = [0.9995*min(ap), 1.0005*max(ap)])
axs[1].set(ylabel = '$e_p$', ylim = [0, 1] , yticks = [0, 0.5 ,1])
axs[2].set(ylabel = r'$i_p$ '+ang_unit     , yticks = angle_yticks)
axs[3].set(ylabel = r'$\Omega_p$ '+ang_unit, yticks = angle_yticks)
axs[4].set(ylabel = r'$\omega_p$ '+ang_unit, yticks = angle_yticks)
axs[5].set(ylabel = r'$M_p$ '+ang_unit     , xlabel = 't (years)', yticks = angle_yticks);

file_plot_pla_fmt = file_plot_pla + '.' + form_plot
if (SAVE_plot):
    if not os.path.exists(dir_plot):
        os.makedirs(dir_plot)   # Create the path if doesn's exists
    plt.savefig(dir_plot + file_plot_pla_fmt, bbox_inches='tight', dpi=int(cal), format=form_plot);  
    print('Saved in:', dir_plot + file_plot_pla_fmt)

plt.show();

In [ ]:
# PARTICLE'S PLOT
fig, axs = plt.subplots(6, 1, figsize = (16, 14), sharex = 'col', sharey = False)

# Plot: a(t), e(t) and angles(t).
for i in range(Npar):
    axs[0].plot(t, a[:][i], marker = '.', ms = 1, ls = 'None')
    axs[1].plot(t, e[:][i], marker = '.', ms = 1, ls = 'None')
    axs[2].plot(t, I[:][i], marker = '.', ms = 1, ls = 'None')
    axs[3].plot(t, O[:][i], marker = '.', ms = 1, ls = 'None')
    axs[4].plot(t, w[:][i], marker = '.', ms = 1, ls = 'None')
    axs[5].plot(t, M[:][i], marker = '.', ms = 1, ls = 'None')

# Labels y ticks:
axs[0].set(ylabel = '$a$ (au)', ylim = [3.967946*0.99, 3.967946*1.01])#, ylim = [0.9995*min(ap), 1.0005*max(ap)])
axs[1].set(ylabel = '$e$', ylim = [0, 1] , yticks = [0, 0.5 ,1])
axs[2].set(ylabel = r'$i$ '+ang_unit     , yticks = angle_yticks)
axs[3].set(ylabel = r'$\Omega$ '+ang_unit, yticks = angle_yticks)
axs[4].set(ylabel = r'$\omega$ '+ang_unit, yticks = angle_yticks)
axs[5].set(ylabel = r'$M$ '+ang_unit, xlabel = 't (years)', yticks = angle_yticks);

file_plot_par_fmt = file_plot_par + '.' + form_plot
if (SAVE_plot):
    plt.savefig(dir_plot + file_plot_par_fmt, bbox_inches='tight', dpi=int(cal), format=form_plot);  
    print('Saved in:', dir_plot + file_plot_par_fmt)

plt.show();

In [ ]:
# OTHER PLOTS (IN PROGRESS)

if (OTROS):
    
    fig, axs = plt.subplots(3, 1, figsize = (16, 7), sharex = 'col', sharey = False)
   
    Kp = 3
    K = 2
    i = 0 # particle index
    
    # Quantities:
    deltaW = w[i]-wp[0]
    theta  = ( K*( M[i]+w[i]+O[i] ) - Kp*( Mp[0]+wp[0]+Op[0]) ) % 360
    sigma  = (theta + (Kp-K)*w[i]) % 360

    if ((ANGLE == 0) or (ANGLE == 1)):
        deltaW = np.where(deltaW > 180, deltaW - 360, deltaW)
        theta = np.where(theta > 180, theta - 360, theta)
        sigma = np.where(sigma > 180, sigma - 360, sigma)

    if ((ANGLE == 2) or (ANGLE == 3)):
        deltaW = np.where(deltaW > pi, deltaW - 2*pi, deltaW)
        theta = np.where(theta > pi, theta - 2*pi, theta)
        sigma = np.where(sigma > pi, sigma - 2*pi, sigma)    
    
    # Gráficos: a(t), e(t) y ángulos(t).
    axs[0].plot(t, deltaW, 's', marker = '.', ms = 1)#, c='orange')
    axs[1].plot(t, theta,  's', marker = '.', ms = 1)#, c='orange')
    axs[2].plot(t, sigma,  's', marker = '.', ms = 1)#, c='orange')

    # Labels y ticks:
    axs[0].set(ylabel = r'$\Delta\varpi$ (°)', yticks = angle_yticks)
    axs[1].set(ylabel = r'$\theta$ (°)', yticks = angle_yticks)
    axs[2].set(ylabel = r'$\sigma$ (°)', xlabel = 't (years)', yticks = angle_yticks);

    file_plot_oth_fmt = file_plot_oth + '_' + str(i) + '.' + form_plot
    if (SAVE_plot):
        plt.savefig(dir_plot + file_plot_oth_fmt , bbox_inches='tight', dpi=int(cal), format=form_plot);  
        print('Saved in:', dir_plot + file_plot_oth_fmt )

    plt.show();

In [ ]:
# DATA SAVING

if (SAVE_data)and(MODO != 2):
    if not os.path.exists(dir_data):
         os.makedirs(dir_data)   # Create the path if doesn's exists
    
    if (EVORB): # FORMAT(F14.3,2f10.4,f9.6,3f7.2,I5)
        formato = "%14.3f %10.4f %10.4f %9.6f %7.2f %7.2f %7.2f %5d"
        nom_cols = "            t       a        e         i     nodo    argper  anomed    N"
    else:
        formato = "%6d %6.2f %8.6f %8.6f %5.2f %5.2f %5.2f %5.2f"
        nom_cols_pla = "     N     t     ap     ep     Ip     Op     wp     Mp     "
        nom_cols_par = "     N     t     a      e      I      O      w      M      "

    if (EVORB):

        # OBJECTS' DATA
        data = np.empty(8)
        for i in range(Nout):
            for j in range(Nobj):

                if (j<Npla):
                    ind = j + 1
                    fila = [ t[i], ap[j][i], ep[j][i], Ip[j][i], Op[j][i], wp[j][i], Mp[j][i], ind ]
                else:
                    ind = j + 10 - Npla
                    j_ = j - Npla
                    fila = [ t[i], a[j_][i], e[j_][i], I[j_][i], O[j_][i], w[j_][i], M[j_][i], ind ]
                
                data = np.vstack([data, fila])
        data = np.delete(data, (0), axis=0)

        np.savetxt(dir_data + file_data_ext, data, fmt=formato, header=nom_cols, comments='')

        
    else:
        
        # PLANET'S DATA
        data_pla = np.empty(8)
        for i in range(Nout):
            for j in range(Npla):
                fila = [ j+1, t[i], ap[j][i], ep[j][i], Ip[j][i], Op[j][i], wp[j][i], Mp[j][i] ]
                data_pla = np.vstack([data_pla, fila])
        data_pla = np.delete(data_pla, (0), axis=0)

        np.savetxt(dir_data + file_data_pla_ext, data_pla, fmt=formato, header=nom_cols_pla, comments='')

        # PARTICLE'S DATA
        data_par = np.empty(8)
        for i in range(Nout):
            for j in range(Npar):
                fila = [ j+1+Npla, t[i], a[j][i], e[j][i], I[j][i], O[j][i], w[j][i], M[j][i] ]
                data_par = np.vstack([data_par, fila])    
        data_par = np.delete(data_par, (0), axis=0)

        np.savetxt(dir_data + file_data_par_ext, data_par, fmt=formato, header=nom_cols_par, comments='')

    print('Saved in:', dir_data)
    
        
# To use in another notebook:    
if (COMPARAR):
    %store ap
    %store ep
    %store Ip
    %store Op
    %store wp
    %store Mp
    %store a
    %store e
    %store I
    %store O
    %store w
    %store M
    %store t
    print(' ')
    
# Last command confirming everything ran ok:
print('Everything seems to have ran succesfully! :)')
print('However, check the results to confirm it.')

In [ ]:
# *** EXTRA INFORMATION ***

# Times:
# Hr          : Hours
# Yr          : Julian years
# Jyr         : Julian years
# Sidereal_yr : Sidereal year
# Yr2pi       : Year divided by 2pi, with year defined as orbital period of planet at 1AU around 1Msun star
# Kyr         : Kiloyears (Julian)
# Myr         : Megayears (Julian)
# Gyr         : Gigayears (Julian)
# Lengths:
# M           : Meters
# Cm          : Centimeters
# Km          : Kilometers
# AU          : Astronomical Units
# Masses:
# Kg          : Kilograms
# Msun        : Solar masses
# Mmercury    : Mercury masses
# Mvenus      : Venus masses
# Mearth      : Earth masses
# Mmars       : Mars masses
# Mjupiter    : Jupiter masses
# Msaturn     : Saturn masses
# Muranus     : Neptune masses
# Mpluto      : Pluto masses

In [ ]:
# TO IMPROVE/ADD:
# 1) Add legend to plots.
# 2) Implement MULTIPLE functionality.
# 3) Better handling of close encounters.
# 4) Add more physical modelling phenomena (Tides, disk drags, radiation pressure, etc.).
# 5) Improve unit system handling.
# 6) Finish other plots/data calcluation functionality.

### THE END ###